## 数据预处理



In [ ]:
import os.path

train_data = open('../data/raw/sogou_rematch/user_tag_query.10W.TRAIN', 'r', encoding='gb18030')
OUTPUT_DATA_PATH = '../data/processed/query_words.train'

try:
    if os.path.exists(OUTPUT_DATA_PATH):
        os.remove(OUTPUT_DATA_PATH)

    output_data = open(OUTPUT_DATA_PATH, 'w', encoding='utf-8')

    for line in train_data:
        line_list = line.split('\t') # 以制表符分割
        line_list = line_list[4:] # 只保留查询词
        output_line = '\n'.join(line_list) # 以换行符连接
        output_data.write(output_line) # 写入文件
finally:
    train_data.close()
    output_data.close()

In [ ]:
import re
import os.path

OUTPUT_DATA_PATH = '../data/processed/cleaned.train'

try:
    train_data = open('../data/processed/query_words.train', 'r', encoding='utf-8')
    if os.path.exists(OUTPUT_DATA_PATH):
        os.remove(OUTPUT_DATA_PATH)

    output_data = open(OUTPUT_DATA_PATH, 'w', encoding='utf-8')

    for line in train_data:
        word_list = line.split('\t')
        pattern = re.compile('(https?|ftp|file)://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]')
        # 如果字符串包含网址，则跳过
        if pattern.search(word_list[0]):
            continue
        line_string = '\n'.join(word_list)
        output_data.write(line_string)
finally:
    train_data.close()
    output_data.close()